## Model Evaluation - Random Forest Models on Diff Data
USA World Series Results,
Run on "Diff" data

# @To Do

- [ ] Look at scaling or normalizing data
- [ ] Look at Neural Networks
- [ ] Resolve issue of target being numeric or string label
- [ ] Explore relationship between Posession Time + Attacking Rucks + Passes

In [217]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [218]:
#Import Data - USA's differential data
df = pd.read_csv('../data/output/new_features_diffdata.csv')
df.head()

#Import validation data
valdf = pd.read_csv('../data/output/new_features_diffdata_validate.csv')
valdf.head()

,Opp,Tournament,Poss_Time_Diff,Score_Diff,Conv_Diff,Tries_Diff,Passes_Diff,Contestable_KO_Win_pct_Diff,PenFK_Against_Diff,RuckMaul_Diff,...,-99 : -75,-74 : -25,-24 : -1,0 : 25,26 : 50,51 : 75,76 : 100,101 : 125,126 : 150,Result
0,Argentina,2018_Paris_7s,47.004608,57.575758,60.000000,-0.250000,53.846154,80.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,1
1,Canada,2018_London_7s,0.467290,17.391304,25.000000,-0.466667,6.329114,20.000000,100.000000,0.000000,...,0.0,0.0,0.0,12.5,0.0,0.0,0.0,0.0,0.0,1
2,England,2018_London_7s,18.994413,37.777778,42.857143,-0.400000,31.147541,26.666667,-33.333333,0.000000,...,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,1
3,England,2018_Paris_7s,18.461538,0.000000,0.000000,0.000000,35.064935,-50.000000,0.000000,-33.333333,...,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,Fiji,2018_Paris_7s,-6.406685,-60.000000,-60.000000,0.000000,22.222222,-25.000000,-100.000000,-100.000000,...,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [219]:
df.info()
list(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 27 columns):
Opp                            156 non-null object
Tournament                     156 non-null object
Poss_Time_Diff                 156 non-null float64
Score_Diff                     156 non-null float64
Conv_Diff                      156 non-null float64
Tries_Diff                     156 non-null float64
Passes_Diff                    156 non-null float64
Contestable_KO_Win_pct_Diff    156 non-null float64
PenFK_Against_Diff             156 non-null float64
RuckMaul_Diff                  156 non-null float64
Ruck_Win_pct_Diff              156 non-null float64
Cards_diff                     156 non-null float64
Lineout_Win_Pct_Diff           156 non-null float64
Scrum_Win_Pct_Diff             156 non-null float64
-175 : -150                    156 non-null float64
-149 : -125                    156 non-null float64
-124 : -100                    156 non-null float64
-99 : -75    

['Opp',
 'Tournament',
 'Poss_Time_Diff',
 'Score_Diff',
 'Conv_Diff',
 'Tries_Diff',
 'Passes_Diff',
 'Contestable_KO_Win_pct_Diff',
 'PenFK_Against_Diff',
 'RuckMaul_Diff',
 'Ruck_Win_pct_Diff',
 'Cards_diff',
 'Lineout_Win_Pct_Diff',
 'Scrum_Win_Pct_Diff',
 '-175 : -150',
 '-149 : -125',
 '-124 : -100',
 '-99 : -75',
 '-74 : -25',
 '-24 : -1',
 '0 : 25',
 '26 : 50',
 '51 : 75',
 '76 : 100',
 '101 : 125',
 '126 : 150',
 'Result']

In [220]:
from sklearn.model_selection import train_test_split

In [221]:
#Drop features that are unneccessary/str or will bias the prediction
rf_data = df.drop((['Opp', 'Score_Diff', 'Tries_Diff','Tournament', 'Conv_Diff','Contestable_KO_Win_pct_Diff']), axis=1)

#Drop rows with Result == "T" (Ties). This label messes up classification models
rf_data.drop(rf_data[rf_data.Result == 2].index, inplace=True)

#Pull out the variable we're trying to predict: 'Result'
X = rf_data.drop('Result',axis=1)
y = rf_data['Result']

In [222]:
rf_data.head()
#Check to insure 'Result' only contains 2 values (W, L)
#rf_data['Result'].describe()
#rf_data.describe()

,Poss_Time_Diff,Passes_Diff,PenFK_Against_Diff,RuckMaul_Diff,Ruck_Win_pct_Diff,Cards_diff,Lineout_Win_Pct_Diff,Scrum_Win_Pct_Diff,-175 : -150,-149 : -125,...,-99 : -75,-74 : -25,-24 : -1,0 : 25,26 : 50,51 : 75,76 : 100,101 : 125,126 : 150,Result
0,13.966480,25.925926,0.000000,0.000000,0.083333,50.0,0.333333,1.0,0.0,0.0,...,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,7.471264,27.868852,-20.000000,-100.000000,0.250000,0.0,-1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,12.5,0.0,0.0,0.0,0.0,0.0,1
2,-33.136095,-10.638298,66.666667,60.000000,-0.550000,0.0,-1.000000,0.0,0.0,0.0,...,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,51.758794,76.119403,-50.000000,-100.000000,0.250000,0.0,0.000000,-1.0,0.0,0.0,...,-37.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,12.880562,38.461538,-33.333333,-33.333333,0.208333,0.0,-1.000000,0.0,0.0,0.0,...,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [223]:
#Pull out the variable we're trying to predict: 'Result'
X = rf_data.drop('Result',axis=1)
y = rf_data['Result']
#X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30)

In [224]:
#Split into train/test/validate sets
#OR, keep as is and use new data for validate
#156 rows in original dataframe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

### Validation set
Imported last two series stops (London & Paris - 12 matches total) to use as a validation set.

Need to incorporate into larger data set later.

In [ ]:
#Split again to create validation set
#Do 66/34 split?
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.34, random_state=1)

## Decision Tree

In [225]:
from sklearn.tree import DecisionTreeClassifier

In [226]:
dtree = DecisionTreeClassifier()

In [227]:
dtree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [228]:
predictions = dtree.predict(X_test)

In [229]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [230]:
#get the model's accuracy score
accuracy_score(y_test, predictions)

0.54347826086956519

In [231]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.50      0.38      0.43        21
          1       0.57      0.68      0.62        25

avg / total       0.54      0.54      0.53        46



In [232]:
print(confusion_matrix(y_test,predictions))

[[ 8 13]
 [ 8 17]]


## Tree Visualization

from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydot 

features = list(rf_data.columns[0:-1])
features

dot_data = StringIO()  
export_graphviz(dtree, out_file=dot_data,feature_names=features,filled=True,rounded=True)

graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph[0].create_png())

## Random Forest

In [234]:
from sklearn.ensemble import RandomForestClassifier

#Fit RF Classifier model
rf = RandomForestClassifier(random_state=101)

from pprint import pprint
# Look at parameters used by our current forest
print('Default Parameters currently in use:\n')
pprint(rf.get_params())

Default Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 101,
 'verbose': 0,
 'warm_start': False}


### Hyperparameters
* n_estimators = number of trees in the foreset
* max_features = max number of features considered for splitting a node
* max_depth = max number of levels in each decision tree
* min_samples_split = min number of data points placed in a node before the node is split
* min_samples_leaf = min number of data points allowed in a leaf node
* bootstrap = method for sampling data points (with or without replacement)

### Random Hyperparameter Grid
Create a parameter grid to sample from during fitting

In [235]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


### Random Search Training
Instantiate the random search and fit it (just as you would any Scikit-Learn model)

In [236]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=101, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=800, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=60, bootstrap=True 
[CV] n_estimators=800, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=60, bootstrap=True 
[CV] n_estimators=800, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=60, bootstrap=True 
[CV] n_estimators=1600, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=False 
[CV]  n_estimators=800, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=60, bootstrap=True, total=   2.0s
[CV] n_estimators=1600, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=False 
[CV]  n_estimators=800, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=60, bootstrap=True, total=   2.1s
[CV] n_estimators=1600, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=F

[CV] n_estimators=1600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=60, bootstrap=False 
[CV]  n_estimators=800, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=50, bootstrap=True, total=   2.6s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False 


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   28.3s


[CV]  n_estimators=1600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=60, bootstrap=False, total=   4.8s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=1600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=60, bootstrap=False, total=   4.9s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=1600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=60, bootstrap=False, total=   4.9s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=100, bootstrap=True 
[CV]  n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False, total=   6.1s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=100, bootstrap=True 
[CV]  n_estima

[CV] n_estimators=1400, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True 
[CV]  n_estimators=1200, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=True, total=   3.7s
[CV] n_estimators=1400, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True 
[CV]  n_estimators=1200, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=True, total=   3.6s
[CV] n_estimators=1200, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=1400, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True, total=   4.5s
[CV] n_estimators=1200, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=1400, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True, total=   4.1s
[CV] n_estimators=1200, 

...

[CV] n_estimators=400, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False 
[CV]  n_estimators=400, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False, total=   2.0s
[CV] n_estimators=400, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False 
[CV]  n_estimators=400, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False, total=   1.5s
[CV] n_estimators=400, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False 
[CV]  n_estimators=400, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False, total=   1.2s
[CV] n_estimators=1600, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=40, bootstrap=True, total=   6.8s
[CV] n_estimators=16

[CV]  n_estimators=1400, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True, total=   4.7s
[CV] n_estimators=800, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=False 
[CV]  n_estimators=1400, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True, total=   4.9s
[CV] n_estimators=800, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=False 
[CV]  n_estimators=1400, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True, total=   4.4s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=40, bootstrap=True 
[CV]  n_estimators=800, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=False, total=   2.4s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=40, bootstrap=True 
[CV]  n_estimators=

[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=40, bootstrap=True, total=   7.1s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=40, bootstrap=True, total=   6.8s
[CV] n_estimators=1400, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=90, bootstrap=False, total=   7.2s
[CV] n_estimators=1400, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=1400, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False, total=   4.3s
[CV] n_estimators=1400

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  5.0min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=101, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

### View the Best Parameters

In [237]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 60,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200}

In [238]:
from sklearn.metrics import accuracy_score

In [239]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 101)
base_model.fit(X_train, y_train)
predictions = base_model.predict(X_test)
#base_accuracy = evaluate(base_model, y_train, y_test)

#Use sklearn.metrics 'accuracy_score' to determine accuracy of trained and test model
base_train_acc = accuracy_score(y_train, base_model.predict(X_train))
base_test_acc = accuracy_score(y_test, predictions)
print("Base Model")
print(base_train_acc)
print(base_test_acc)

#Use best paramaters from RandomSearchCV
best_random = rf_random.best_estimator_

#fit model with best parameters
best_random.fit(X_train, y_train)

#get predictions with best parameters
random_predict = best_random.predict(X_test)

random_train_acc = accuracy_score(y_train, best_random.predict(X_train))
random_test_acc = accuracy_score(y_test, random_predict)
print("\n")
print("Randomized Search Model")
print(random_train_acc)
print(random_test_acc)

Base Model
1.0
0.45652173913


Randomized Search Model
0.971428571429
0.54347826087


**Results**
```
Base Model
1.0
0.45652173913


Randomized Search Model
0.971428571429
0.54347826087
```

See http://scikit-learn.org/stable/modules/model_evaluation.html

In [240]:
# Create a confusion matrix
from sklearn.metrics import confusion_matrix

print("Confusion Matrix")
print(confusion_matrix(y_test, random_predict))

Confusion Matrix
[[ 8 13]
 [ 8 17]]


In [241]:
# Create a classification report

from sklearn.metrics import classification_report

#import libraries to ignore UndefinedMetricWarning
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

#get the model's accuracy score
accuracy_score(y_test, random_predict)
print("\n")
print("Classification Report")
print(classification_report(y_test, random_predict))

#print accuracy score
print("\n")
print("Accuracy Score")
print(best_random.score(X_test, y_test))



Classification Report
             precision    recall  f1-score   support

          0       0.50      0.38      0.43        21
          1       0.57      0.68      0.62        25

avg / total       0.54      0.54      0.53        46



Accuracy Score
0.54347826087


**Randomized Search Results**
Randomized Search results in a 54% accuracy score, about 8% better than the base model.

### Grid Search with Cross Validation
Random search allowed us to narrow down the range for each hyperparameter. Now that we know where to concentrate our search, we can explicitly specify every combination of settings to try. We do this with GridSearchCV, a method that, instead of sampling randomly from a distribution, evaluates all combinations we define. 

**Original**
```
{'bootstrap': True,
 'max_depth': 100,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 1000}
```

### Latest
```
{'bootstrap': True,
 'max_depth': 60,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200}
```

To use Grid Search, we make another grid based on the best values provided by random search:

In [243]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [60, 80, 90, 100, 110],
    'max_features': [2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'min_samples_split': [2, 5, 8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a base model
#rf = RandomForestClassifier(random_state = 101)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [244]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

best_grid = grid_search.best_estimator_
#grid_accuracy = evaluate(best_grid, X_test, y_test)

print("Base Model")
print(base_train_acc)
print(base_test_acc)

#get predictions with best parameters
grid_predict = best_grid.predict(X_test)

grid_train_acc = accuracy_score(y_train, best_grid.predict(X_train))
grid_test_acc = accuracy_score(y_test, grid_predict)
print("\n")
print("Grid Search Model")
print(grid_train_acc)
print(grid_test_acc)

#print('Improvement of {:0.2f}%.'.format( 100 * (grid_test_acc - base_test_acc) / base_test_acc))

Fitting 3 folds for each of 2000 candidates, totalling 6000 fits
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   10.0s


[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=10, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=10, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=10, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=10, n_estimators=200 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=10, n_estimators=200, total=   0.5s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=1, min_samples_split=10, n_estimators=200 
[CV]  bootstrap=True, max_depth=60, max_features=2, 

[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=2, n_estimators=300, total=   1.1s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=5, n_estimators=100 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=5, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=5, n_estimators=100 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=5, n_estimators=100, total=   0.4s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=5, n_estimators=100 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=5, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=5, n_estimators=200 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samp

[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=10, n_estimators=300, total=   1.1s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=10, n_estimators=1000 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=10, n_estimators=300, total=   1.1s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=12, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=12, n_estimators=100, total=   0.5s
[CV] bootstrap=True, max_depth=60, max_features=2, min_samples_leaf=2, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=60, max_features=2,

...

[CV] bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=4, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=4, min_samples_split=12, n_estimators=200, total=   0.7s
[CV] bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=4, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=4, min_samples_split=12, n_estimators=200, total=   0.7s
[CV] bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=4, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=4, min_samples_split=10, n_estimators=1000, total=   3.6s
[CV] bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=4, min_samples_split=12, n_estimators=300 
[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=4, min_samples_split=10, n_estimators=1000, total=   3.6s
[CV] bootstrap=True, max_depth=110, max_fe

[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=8, n_estimators=100, total=   0.4s
[CV] bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=8, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=8, n_estimators=200, total=   0.7s
[CV] bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=5, n_estimators=1000, total=   3.6s
[CV] bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=5

[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=12, n_estimators=1000, total=   2.9s
[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=12, n_estimators=1000, total=   2.7s
[CV]  bootstrap=True, max_depth=110, max_features=5, min_samples_leaf=5, min_samples_split=12, n_estimators=1000, total=   2.6s


[Parallel(n_jobs=-1)]: Done 6000 out of 6000 | elapsed: 38.7min finished


Base Model
1.0
0.45652173913


Grid Search Model
1.0
0.565217391304


In [245]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_test_acc - base_test_acc) / base_test_acc))

Improvement of 23.81%.


### Grid Search Accuracy Results
**Base Model**
```
1.0
0.45652173913
```
**Grid Search Model**
```
1.0
0.565217391304
```

***Improvement of 23.81%. to 56.5%***

In [246]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 110,
 'max_features': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 300}

In [247]:
# examine the best model#
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=110, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.704761904762
{'bootstrap': True, 'max_depth': 110, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


**Best Estimator**
```
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=110, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
```

**Best Score**
```
0.704761904762
```
**Best Parameters**
```
{'bootstrap': True, 'max_depth': 110, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
```

### Use new parameters from gridsearch to create and fit model

In [248]:
#Fit classifier with new model parameters from gridsearch
rfc = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=110, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

#Fit model
rfc.fit(X, y)

#Predict Classifier
rfc_pred = rfc.predict(X_test)

## Random Forest Model Eval

In [249]:
#Accuracy
rfc_acc = accuracy_score(y_test, rfc_pred)
print(rfc_acc)

1.0


In [250]:
#Find Feature Importances
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

print("Feature Importance")
print(feature_importances)

Feature Importance
                      importance
Poss_Time_Diff          0.176322
Passes_Diff             0.148887
Ruck_Win_pct_Diff       0.135487
PenFK_Against_Diff      0.113932
RuckMaul_Diff           0.090475
Lineout_Win_Pct_Diff    0.080941
Scrum_Win_Pct_Diff      0.069687
Cards_diff              0.043888
0 : 25                  0.032440
-74 : -25               0.030564
51 : 75                 0.026398
26 : 50                 0.022696
-24 : -1                0.009982
76 : 100                0.005788
-99 : -75               0.004198
-124 : -100             0.003239
-175 : -150             0.002819
126 : 150               0.002257
-149 : -125             0.000000
101 : 125               0.000000


In [251]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

#Output confusion matrix
print("Confusion Matrix")
print(confusion_matrix(y_test,rfc_pred))

#import libraries to ignore UndefinedMetricWarning
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

#get the model's accuracy score
accuracy_score(y_test, rfc_pred)
print("\n")
print("Classification Report")
print(classification_report(y_test,rfc_pred))

#print accuracy score
print("\n")
print("Accuracy Score")
print(rfc.score(X_test, y_test))

Confusion Matrix
[[21  0]
 [ 0 25]]


Classification Report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        21
          1       1.00      1.00      1.00        25

avg / total       1.00      1.00      1.00        46



Accuracy Score
1.0


## Predict on Validation Set

In [252]:
#Drop features that are unneccessary/str or will bias the prediction
val_data = valdf.drop((['Opp', 'Score_Diff', 'Tries_Diff','Tournament', 'Conv_Diff','Contestable_KO_Win_pct_Diff']), axis=1)

#Drop rows with Result == "T" (Ties). This label messes up classification models
val_data.drop(val_data[val_data.Result == 2].index, inplace=True)

#Pull out the variable we're trying to predict: 'Result'
val_X = val_data.drop('Result',axis=1)
val_y = val_data['Result']

In [254]:
#Run Prediction Classifier on validation data
rfc_val_pred = rfc.predict(val_X)

In [255]:
#Accuracy
rfc_val_acc = accuracy_score(val_y, rfc_val_pred)
print(rfc_val_acc)

0.9
